<div style="color:white;background-color:#0B244E;padding:20px;width:100%">
<img src="https://www.eseg.edu.br/assets/dist/assets/images/logo/136x136.svg" style=""></img>
</div>
<div style="color:white;background-color:#0B244E;padding:20px;width:100%;border-top: 1px solid white">
Data Science 1 / Prof. Fernando Pablos
</div>

<h1>Análise Exploratória e limpeza de dados (EDA)</h1>

<p>Como já falado na parte teórica, nem sempre nossos dados chegam para nós da forma ideal, para isso precisamos explorá-los e, se necessário, corrigí-los antes de realizarmos nossas análises.</p>
<p>Vamos começar carregando nossos dados como já aprendemos em aulas anteriores:</p>




In [ ]:
import pandas as pd

df = pd.read_excel('eda.xlsx')
df.head()

<p>Podemos observar alguns problemas no nome de nossas colunas... veja, dt1, dt5 e dt30, o que será que são essas colunas?</p>
<p style="color:red">dt1 parece ser o nome da empresa, dt5 sua área de atuação e dt30 seu endereço, vamos corrigir.</p>

In [ ]:
df = df.rename(columns={'dt1': 'Nome da empresa', 'dt5': 'Área de Atuação', 'dt30': 'Endereço'})
df.head()

In [ ]:
df.info()

<p style="color:red;">Existem algumas células vazias, o que será que podemos fazer com elas? Temos algumas alternativas.</p>


In [ ]:
#Para as células categóricas, podemos substituir pela moda
area_atuacao_moda = df['Área de Atuação'].mode()
df['Área de Atuação'] = df['Área de Atuação'].fillna(area_atuacao_moda[0])


tipo_moda = df['Tipo:'].mode()
df['Tipo:'] = df['Tipo:'].fillna(tipo_moda[0])

#Para as células numéricas, podemos pegar a mediana

lucro_mediana = df['Lucro Líquido Arredondado (R$)'].median()
df['Lucro Líquido Arredondado (R$)'] = df['Lucro Líquido Arredondado (R$)'].fillna(lucro_mediana)

ano_mediana = df['Ano de Fundação'].median()
df['Ano de Fundação'] = df['Ano de Fundação'].fillna(ano_mediana)

df.info()



<p>Tudo corrigido!</p>
<h2>Analisando dados categóricos</h2> 

<p>Um jeito interessante de analisar dados categóricos, é observar a contagem  e também gráficos de barras.</p>

In [ ]:
#Vamos começar analisando o nome das empresas

grupo_nome_empresa = df.groupby(['Nome da empresa']).size()
print(grupo_nome_empresa) #Visão de relatório
grupo_nome_empresa.plot.bar() #Visão de gráfico

<p style="color:red">Podemos observar que algumas empresas estão sendo repetidas duas ou mais vezes, isso é um problema</p>

In [ ]:
#Irá nos mostrar os registros duplicados
df[df.duplicated(['Nome da empresa'])]

In [ ]:
#Excluindo os duplicados, mantendo o primeiro registro

df = df.drop_duplicates(subset="Nome da empresa", keep='first')

#Mostrando novamente se ainda há registros duplicados
df[df.duplicated(['Nome da empresa'])]

In [ ]:
#Vamos ver agora os bairros, se há algo errado
grupo_bairro = df.groupby(['bairro']).size()
print(grupo_bairro)
grupo_bairro.plot.bar()

Parece estar tudo bem com essa coluna. Podemos ter várias empresas no mesmo bairro.

In [ ]:
#Vamos ver a área de atuação da empresa

grupo_atuacao = df.groupby('Área de Atuação').size()
print(grupo_atuacao)
grupo_atuacao.plot.bar()

<p style="color:red">A área de atuação 'Teste' parece ser um erro de cadastro.</p>

In [ ]:
#Não vamos manter esses dados, pois podem impactar na análise erroneamente
df = df[df['Área de Atuação'] != 'Teste']

#Vamos ver agora se ainda aparecem:

grupo_atuacao = df.groupby('Área de Atuação').size()
print(grupo_atuacao)
grupo_atuacao.plot.bar()

Problema resolvido!

In [ ]:
#Vamos observar os CNPJ's

grupo_cnpj = df.groupby('cnpj').size()
print(grupo_cnpj)
grupo_cnpj.plot.bar()

Não temos CNPJ's repetidos, então não precisamos tratar essa coluna.

In [ ]:
#Vamos ver os estados

grupo_estado = df.groupby('Estado').size()
print(grupo_estado)
grupo_estado.plot.bar()

<p style="color:red">Os estados XX e ZZ não existem.</p>

In [ ]:
#Para resolver, vamos filtrar usando o pandas, removendo esses dados

df = df[(df['Estado'] != 'XX') & (df['Estado'] != 'ZZ')]

#E observamos se o problema persiste
grupo_estado = df.groupby('Estado').size()
print(grupo_estado)
grupo_estado.plot.bar()

Resolvido!

In [ ]:
#Vamos ver os tipos

grupo_tipo = df.groupby('Tipo:').size()
print(grupo_tipo)
grupo_tipo.plot.bar()

<p style="color:red">Temos tipos que deveriam ser o mesmo com nomes diferentes</p>

In [ ]:
# Vamos localizar esses registros e substituí-los, para padronizarmos 
# nossos dados

df['Tipo:'] = df['Tipo:'].replace({
    'Lemitada': 'Ltda',
    'Limitada': 'Ltda',
    'Microempreendedor Individual': 'MEI',
    'Sociedade Anônima': 'SA'
})

#Vamos ver se deu certo
grupo_tipo = df.groupby('Tipo:').size()
print(grupo_tipo)
grupo_tipo.plot.bar()

<h2>Analisando dados numéricos</h2> 

In [ ]:
import seaborn as sns
pd.set_option('display.float_format', '{:.2f}'.format)  # Desativando a notação científica
#Vamos analisar o lucro líquido acumulado

df['Lucro Líquido Arredondado (R$)'].describe()


<p style="color: red">O mínimo parece ser muito baixo e o máximo muito alto, podemos verificar pontualmente esses valores e ver se não há mais casos.</p>

In [ ]:
sns.boxplot(df['Lucro Líquido Arredondado (R$)'])

In [ ]:
sns.histplot(df['Lucro Líquido Arredondado (R$)'])

In [ ]:
#Observamos que existem valores exagerados, vamos tentar removê-los

df = df[(df['Lucro Líquido Arredondado (R$)'] < 1000000000)]

sns.boxplot(df['Lucro Líquido Arredondado (R$)'])


In [ ]:
#Ainda parece que temos um valor exagerado negativo, vamos removê-lo também
df = df[df['Lucro Líquido Arredondado (R$)'] > -10000000]

sns.boxplot(df['Lucro Líquido Arredondado (R$)'])

Aparentemente nossos dados estão corretos agora.

In [ ]:
sns.histplot(df['Lucro Líquido Arredondado (R$)'])

Vamos ver se para o ano há algum problema

In [ ]:
df['Ano de Fundação'].describe()

In [ ]:
sns.histplot(df['Ano de Fundação'])

In [ ]:
sns.boxplot(df['Ano de Fundação'])

<p style="color: red">Bom, considerando que nossa base seja de um sistema atual, parece estranho ter empresas fundadadas antes dos anos 1800 e também não chegamos no ano 2900. Vamos remover.</p>

In [ ]:
df = df[(df['Ano de Fundação'] >= 1800) & (df['Ano de Fundação'] <= 2025)]
sns.histplot(df['Ano de Fundação'])

In [ ]:
sns.boxplot(df['Ano de Fundação'])

Para encerrar, vamos ver se há alguma correlação entre o ano de fundação e o lucro líquido e identificar se há outliers

In [ ]:
sns.scatterplot(data=df, x=df['Ano de Fundação'], y=df['Lucro Líquido Arredondado (R$)'])

In [ ]:
#Vamos analisar uma matriz de correlação também
df_correlacao = df[['Lucro Líquido Arredondado (R$)', 'Ano de Fundação']]
correlacao = df_correlacao.corr()
display(correlacao)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(
    correlacao,
    annot=True,       # Mostra os valores dentro dos quadrados
    cmap='coolwarm',  # Paleta de cores (azul/vermelho)
    vmin=-1,          # Valor mínimo da escala
    vmax=1,           # Valor máximo da escala
    linewidths=0.5
)
plt.title('Matriz de Correlação')
plt.show()

Não há correlação e, portanto, não conseguimos identificar outliers.

<p style="color: green">Agora nossa base está limpa para realizarmos as análises, ou para darmos início ao aprendizado de máquina.</p>